# Progetto BigData
## Valerio Di Zio e Francesco Magnani

intro here

In [1]:
%%configure -f
{"executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [2]:
val bucketname = "unibo-bd2324-fmagnani"

val path_book_sample = "s3a://"+bucketname+"/project/book_sample134.json"
val path_book_sample32row = "s3a://"+bucketname+"/project/book_sample32row.json"
val path_positive_words = "s3a://"+bucketname+"/project/positive-words.txt"
val path_negative_words = "s3a://"+bucketname+"/project/negative-words.txt"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1708096619670_0003,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2324-fmagnani
path_book_sample: String = s3a://unibo-bd2324-fmagnani/project/book_sample134.json
path_book_sample32row: String = s3a://unibo-bd2324-fmagnani/project/book_sample32row.json
path_positive_words: String = s3a://unibo-bd2324-fmagnani/project/positive-words.txt
path_negative_words: String = s3a://unibo-bd2324-fmagnani/project/negative-words.txt
res3: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1708096619670_0003/


In [3]:
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.DataFrame
import org.apache.spark.broadcast.Broadcast

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.rdd.RDD
import org.apache.spark.sql.DataFrame
import org.apache.spark.broadcast.Broadcast


In [4]:
case class BookReview(
    id: String,
    overall:Double,
    reviewTime:String,
    reviewerID:String,
    asin:String,
    reviewText:String,
    summary:String
)

object BookReview{
    def extract(row:org.apache.spark.sql.Row) = {
        val overall = row.getAs[Double]("overall")
        val reviewTime = row.getAs[String]("reviewTime")
        val reviewerID = row.getAs[String]("reviewerID")
        val asin = row.getAs[String]("asin")
        val reviewText = row.getAs[String]("reviewText")
        val summary = row.getAs[String]("summary")
        val id = reviewerID + "-" + asin
    
        new BookReview(id, overall, reviewTime, reviewerID, asin, reviewText, summary)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class BookReview
defined object BookReview
Companions must be defined together; you may wish to use :paste mode for this.


## Caricamento del Dataset

In [47]:
val bookSampleRdd32row = spark.read.option("mode", "DROPMALFORMED").json(path_book_sample).map(BookReview.extract)
//val bookSampleRdd32row = spark.read.option("mode", "DROPMALFORMED").json(path_book_sample32row).map(BookReview.extract)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bookSampleRdd32row: org.apache.spark.sql.Dataset[BookReview] = [id: string, overall: double ... 5 more fields]


# Job 1 - Sentiment Analysis 

In [ ]:
val positiveWords = sc.textFile(path_positive_words)
val negativeWords = sc.textFile(path_negative_words)

In [ ]:
def splitAndClean(id:String, text: String): Array[(String, String, Int)] = 
    text.split("\\s+").map(el => (id, el.toLowerCase.replaceAll("[^a-zA-Z\\s]", ""), 1))

# Tentativi di ottimizzazione
1. utilizzo di variabili broadcast per le liste di parole uguali per tutti
2. ripartizione dei dataset per ottimizzare il join

# Prima fase del Job 1

Recensioni divise in parole tramite flat map, dopodiché join con parole positive e negative.

In [ ]:
def countOccurrenciesOfWordReview(reviews: DataFrame): DataFrame = 
    reviews.
        groupBy("id", "word").
        agg(sum("occurrencies").alias("occurrencies"))


def joinAndMapWords(review:DataFrame, wordsDataset: Broadcast[Array[String]]): DataFrame = {
    val wordDF = review.sparkSession.createDataFrame(wordsDataset.value.map(Tuple1.apply)).toDF("word")
    
    val aggregRevDF = countOccurrenciesOfWordReview(review)
    
    return aggregRevDF.
        join(wordDF, aggregRevDF("word") === wordDF("word")).
        groupBy("id").
        agg(sum("occurrencies").alias("occurrencies"))
}

    

val reviewsDataset = bookSampleRdd32row.
    select("id", "reviewText").
    dropDuplicates(Seq("id")).
    filter(review => review.getAs[String]("reviewText") != null && review.getAs[String]("reviewText").length > 3).
    flatMap(review => splitAndClean(review.getAs[String]("id"), review.getAs[String]("reviewText"))).
    filter(row => row._2.length > 3).
    toDF("id", "word", "occurrencies").
    cache()

val positiveWordsBroadcast = sc.broadcast(positiveWords.collect())
val negativeWordsBroadcast = sc.broadcast(negativeWords.collect())


val withPositiveDF = joinAndMapWords(reviewsDataset, positiveWordsBroadcast)
    
val withNegativeDF = joinAndMapWords(reviewsDataset, negativeWordsBroadcast)


val reducedResult = withPositiveDF.
    join(withNegativeDF, Seq("id"), "outer").
    toDF("id", "valuePositive", "valueNegative").
    withColumn("estimated_overall", coalesce(col("valuePositive"), lit(0L)) - coalesce(col("valueNegative"), lit(0L)))
     

# Seconda fase del Job 1 - Calcolo risultati

Qui calcoliamo i risultati utilizzando una normalizzazione Z-Score, successivamente normalizzata a sua volta tramite Min-Max tra 1 e 5, in modo da confrontarli con i voti delle recensioni stesse.

In [ ]:
val resultDataframe = reducedResult.select("id", "estimated_overall").cache()

val minValue = resultDataframe.agg(min("estimated_overall")).head().getLong(0).toDouble
val maxValue = resultDataframe.agg(max("estimated_overall")).head().getLong(0).toDouble


// Calculate mean and standard deviation

val meanAndStdDF = resultDataframe.agg(mean("estimated_overall"), stddev("estimated_overall")).collect()
val meanValue = meanAndStdDF(0)(0).asInstanceOf[Double]
val stdValue = meanAndStdDF(0)(1).asInstanceOf[Double]

// Normalize the data using Z-score normalization formula

val normalizedDF = resultDataframe.withColumn("z_score", (col("estimated_overall") - meanValue) / stdValue)

// Scale the normalized values to be between -1 and 1

val minMaxDF = normalizedDF.agg(min("z_score"), max("z_score")).collect()
val minZScore = minMaxDF(0)(0).asInstanceOf[Double]
val maxZScore = minMaxDF(0)(1).asInstanceOf[Double]


val normalizedAndScaledDF = normalizedDF.withColumn("estimated_overall", 
                                                    (col("z_score") - minZScore) / (maxZScore - minZScore) * 4 + 1).
    select("id", "estimated_overall")

// DEPRECATED
// a = min, b = max, c = 1 e d = 5
def mapIntToOverallVote(toMap: Double, fromMin: Double, fromMax: Double, toMin:Double=1, toMax:Double=5): Double =
     toMin + ((toMax-toMin)/(fromMax-fromMin))*(toMap-fromMin)
    

In [ ]:
val relativeErrors = 
bookSampleRdd32row.
    map(review => (review.id, review.overall)).
    toDF("id", "real_value").
    join(normalizedAndScaledDF, Seq("id")).
    withColumn("relative_error", abs(coalesce(col("real_value"), lit(3.0)) - 
                                     coalesce(col("estimated_overall"), lit(3.0))) / 
                                   coalesce(col("real_value"), lit(3.0)))

In [ ]:
val sumOfRelativeErrors = relativeErrors.select("relative_error").agg(sum("relative_error")).
                                collect()(0).getDouble(0)
val meanError =  sumOfRelativeErrors / relativeErrors.count

println("Errore Medio: " + meanError*100 + "%")

# Job 2 - Frequent Pattern Mining

Qui inizia il secondo Job, dove effettuiamo un Frequent Pattern Mining utilizzando le funzionalità di Spark.

## Creazione del DataFrame delle reviews
Prima di tutto andiamo a definire una funzione di utility che ci permette di eliminare eventuali caratteri estranei dai testi delle recensioni (es. virgole, punti etc.). 

Dopodiché, utilizziamo il Dataset caricato anche per il primo Job e ne prendiamo solo le colonne `"id"` e `"reviewText"`, sono le uniche che ci servono. In modo simile al primo Job, eliminiamo le recensioni duplicate usando ```dropDuplicates```: questo è necessario perché esplorando il Dataset sono state trovate recensioni identiche per lo stesso prodotto, facenti riferimento a diverse versioni dello stesso.

Per comodità, il Dataset viene convertito in un RDD in modo da svolgere le successive operazioni di **filter** e **map**. Infine, otteniamo il risultato in forma di DataFrame.

In [6]:
def clean(text: String): String =
    text.toLowerCase.replaceAll("[^a-zA-Z\\s]", " ")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

clean: (text: String)String


In [48]:
val reviewsDF = bookSampleRdd32row.
    select("id", "reviewText").
    dropDuplicates(Seq("id")).
    map(review => (review.getAs[String]("id"), review.getAs[String]("reviewText"))).
    rdd.
    filter({case (_, text) => text != null && text.length > 3}).
    map({case (id, text) => (id, clean(text))}).
    toDF("id", "text").
    cache()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

reviewsDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, text: string]


## Creazione di Tokenizer e StopWord remover

Queste due funzioni sono utili per **dividere i testi in parole** e **rimuovere le stop words** (parole comunemente utilizzate in un linguaggio ma normalmente prive di valore se prese da sole, per esempio "is", "the" etc.)

In [8]:
import org.apache.spark.ml.feature.{Tokenizer, StopWordsRemover}

val tokenizer = new Tokenizer().
    setInputCol("text").
    setOutputCol("words")

val remover = new StopWordsRemover().
    setInputCol("words").
    setOutputCol("filtered_words")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.{Tokenizer, StopWordsRemover}
tokenizer: org.apache.spark.ml.feature.Tokenizer = tok_3ec197d3b704
remover: org.apache.spark.ml.feature.StopWordsRemover = StopWordsRemover: uid=stopWords_c2335289e0fa, numStopWords=181, locale=en_US, caseSensitive=false


## Pulizia del Dataset

Nella cella seguente andiamo a utilizzare il `tokenizer` e il `remover` creati precedentemente per pulire ulteriormente il DataFrame. 

Osservando i risultati di `filteredWordsDF` si può notare che sono ancora presenti refusi e stringhe contenenti spazi. Oltre a questo, la classe che verrà utilizzata successivamente, ovvero `FPGrowth`, richiede che all'interno di ogni pattern siano presenti stringhe **univoche**, motivo per cui andiamo a svolgere l'operazione `toArray.distinct`, che rimuove elementi duplici all'interno di un'Array di Scala)

Otteniamo, infine, un DataFrame da una sola colonna, contenente un Array di parole per ciascuna recensione. È possibile notare che non sono più presenti gli ID: questi infatti non sono più necessari al fine di computare i Frequent Pattern.

In [49]:
import scala.collection.mutable.WrappedArray

val wordsDF = tokenizer.transform(reviewsDF)
val filteredWordsDF = remover.transform(wordsDF)

val uniqueWordsDF = filteredWordsDF.
    select("filtered_words").
    distinct().
    map(item => item.getAs[WrappedArray[String]](0).filter(_.length > 1).toArray.distinct).
    toDF("words")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import scala.collection.mutable.WrappedArray
wordsDF: org.apache.spark.sql.DataFrame = [id: string, text: string ... 1 more field]
filteredWordsDF: org.apache.spark.sql.DataFrame = [id: string, text: string ... 2 more fields]
uniqueWordsDF: org.apache.spark.sql.DataFrame = [words: array<string>]


## Utilizzo di FPGrowth

Nella cella seguente definiamo una funzione che ci permette di utilizzare facilmente la classe di `FPGrowth` per calcolare i Frequent Pattern di un DataFrame. Questa istanzia la classe in questione, fissando gli iper-parametri a dei valori arbitrari: in particolare, il **supporto minimo** per considerare un pattern *frequent* e la **confidenza minima**, ovvero (dalla documentazione di Spark) un'indicazione di quanto spesso una *association rule* viene trovata vera. Quest'ultima non influenza la ricerca dei Frequent Pattern.

In [50]:
import org.apache.spark.ml.fpm.FPGrowth

def frequentItemsets(df: DataFrame): DataFrame = {
    val fpg = new FPGrowth().
        setItemsCol("words").
        setMinSupport(0.1).
        setMinConfidence(0.5)

    val model = fpg.fit(df)
    return model.freqItemsets
}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.fpm.FPGrowth
frequentItemsets: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


## Visualizzazione dei risultati

Qui effettuiamo una prima visualizzazione dei risultati, mostrandone 50. È possibile notare fin da subito come i risultati più frequenti siano parole come "libro" e "leggere", risultati piuttosto predicibili. 

In [ ]:
frequentItemsets(uniqueWordsDF).show(50)

model.transform(uniqueWordsDF).show()

## Secondo Dataset, rimuovendo parole triviali (es. "book", "read" etc.)

Per eliminare queste parole dalla ricerca, effettuiamo un'ultima modifica al nostro DataFrame, eliminando parole triviali come "libro" e "leggere", prese da un file opportunamente creato per lo scopo. Questo viene poi tramutato in un array contenuto in una **broadcast variable**. Infine, ogni array di parole viene modificato in modo da sottrarre queste parole alla ricerca.


In [51]:
// caricamento della blacklist delle parole da escludere
val path_blacklist_words = "s3a://"+bucketname+"/project/blacklist.txt"

val blacklistWords = sc.broadcast(sc.textFile(path_blacklist_words).collect)

val prunedDF = uniqueWordsDF.
    map(item => item.getAs[WrappedArray[String]](0).toArray.filterNot(blacklistWords.value.contains)).
    toDF("words")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path_blacklist_words: String = s3a://unibo-bd2324-fmagnani/project/blacklist.txt
blacklistWords: org.apache.spark.broadcast.Broadcast[Array[String]] = Broadcast(122)
prunedDF: org.apache.spark.sql.DataFrame = [words: array<string>]


## Secondo utilizzo di FPGrowth

Il risultato finale appare più interessante. Questo potrebbe essere ulteriormente migliorato escludendo più parole dalla ricerca.  

In [53]:
frequentItemsets(prunedDF).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|       items| freq|
+------------+-----+
|     [story]|34175|
|     [great]|32786|
|      [good]|29632|
|      [like]|27869|
|      [time]|25746|
|[characters]|23769|
|      [love]|22455|
|      [well]|22368|
|     [first]|22238|
|      [much]|21531|
|    [series]|20257|
|      [many]|18528|
|    [really]|18458|
|       [get]|17267|
+------------+-----+

